In [1]:
import json

In [8]:
with open("../can_db.json", "rb") as f:
    ex = json.loads(f.read())

In [10]:
ex.keys()

dict_keys(['messages'])

In [12]:
ex["messages"][0]

{'id': 768,
 'is_extended_frame': False,
 'name': 'LightBus',
 'signals': [{'bit_length': 8,
   'factor': '1',
   'is_big_endian': True,
   'is_float': False,
   'is_signed': False,
   'name': 'CTFLights',
   'offset': '0',
   'start_bit': 16}]}

In [19]:
def gen_sig(start_bit, bit_length):
    return {'bit_length': bit_length,
     'factor': '1',
     'is_big_endian': True,
     'is_float': False,
     'is_signed': False,
     'name': 'OneUp',
     'offset': '0',
     'start_bit': start_bit
    }

In [20]:
gen_sig(8, 0)

{'bit_length': 0,
 'factor': '1',
 'is_big_endian': True,
 'is_float': False,
 'is_signed': False,
 'name': 'OneUp',
 'offset': '0',
 'start_bit': 8}

In [15]:
ex["messages"][1]["signals"][0]

{'bit_length': 8,
 'factor': '1',
 'is_big_endian': True,
 'is_float': False,
 'is_signed': False,
 'name': 'OneUp',
 'offset': '0',
 'start_bit': 0}

In [21]:
import glob
logs = glob.glob("*.log")
logs

['candump-2020-08-06_130024.log',
 'candump-2020-08-06_124354.log',
 'candump-2020-08-06_124435.log',
 'hs_long.log',
 'i_can.log',
 'ican_dump.log',
 'hs_odb_probe.log',
 'lock_unlock.log',
 'candump-2020-08-06_132427.log']

In [22]:
class CanMsg(object):
    def __init__(self, canid, data):
        self.canid=canid
        self.data=data
        
    def __repr__(self):
        return f"CAN<{self.canid}, {self.data}>"
def parse_log(log_file):
    messages = list()
    with open(log_file, "r") as f:
        for can_msg in f.readlines():
            can_id, can_data = can_msg.strip().split(" ")[-1].split("#")
            cm = CanMsg(can_id, can_data)
            messages.append(cm)
    #
    unique_datas = dict()
    #
    for canid in set([msg.canid for msg in messages]):
        id_datas = set([msg.data for msg in messages if msg.canid==canid])
        unique_datas[canid] = id_datas
    #
    return unique_datas

In [24]:
data = parse_log('candump-2020-08-06_132427.log')

In [25]:
data.keys()

dict_keys(['511', '361', '405', '2A7', '060', '080', '2B0', '1A4', '150', '504', '241', '400', '415', '255', '3A0', '1E0', '2C0', '040', '160', '290', '265', '070', '2A0', '300', '360', '230', '110', '370', '310', '2F5', '020', '100', '541', '508', '1B0', '270', '108', '250', '240', '090', '1A0', '501', '083', '030', '0A0', '581', '2E0', '03A', '320', '1C0', '340', '1A8', '40A', '220'])

In [27]:
for start_bit in range(0, 8, 64):
    print(gen_sig(bit_length=8, start_bit=start_bit))

{'bit_length': 8, 'factor': '1', 'is_big_endian': True, 'is_float': False, 'is_signed': False, 'name': 'OneUp', 'offset': '0', 'start_bit': 0}


In [103]:
def gen_dbc_entry(can_id):
    dbc_entry = {'id': int(f"{can_id}", 16),
     'is_extended_frame': False,
     'name': f'Ox{can_id}',
     'signals': list()}

    def gen_sig(start_bit, bit_length=8):
        return {'bit_length': bit_length,
         'factor': "1",
         'is_big_endian': True,
         'is_float': False,
         'is_signed': False,
         'name': f'Ox{can_id}_{start_bit}',
         'offset': "0",
         'start_bit': start_bit
        }
    
    for start_bit in range(0, 64, 8):
        sig = gen_sig(start_bit=start_bit)
        dbc_entry["signals"].append(sig)
        
    return dbc_entry

In [104]:
gen_dbc_entry(511)

{'id': 1297,
 'is_extended_frame': False,
 'name': 'Ox511',
 'signals': [{'bit_length': 8,
   'factor': '1',
   'is_big_endian': True,
   'is_float': False,
   'is_signed': False,
   'name': 'Ox511_0',
   'offset': '0',
   'start_bit': 0},
  {'bit_length': 8,
   'factor': '1',
   'is_big_endian': True,
   'is_float': False,
   'is_signed': False,
   'name': 'Ox511_8',
   'offset': '0',
   'start_bit': 8},
  {'bit_length': 8,
   'factor': '1',
   'is_big_endian': True,
   'is_float': False,
   'is_signed': False,
   'name': 'Ox511_16',
   'offset': '0',
   'start_bit': 16},
  {'bit_length': 8,
   'factor': '1',
   'is_big_endian': True,
   'is_float': False,
   'is_signed': False,
   'name': 'Ox511_24',
   'offset': '0',
   'start_bit': 24},
  {'bit_length': 8,
   'factor': '1',
   'is_big_endian': True,
   'is_float': False,
   'is_signed': False,
   'name': 'Ox511_32',
   'offset': '0',
   'start_bit': 32},
  {'bit_length': 8,
   'factor': '1',
   'is_big_endian': True,
   'is_float':

In [105]:
dbc = {
    "messages": list()
}
for can_id in data.keys():
    dbc["messages"].append(gen_dbc_entry(can_id))

with open("spewed_dbc.json", "w") as fp:
    json.dump(obj=dbc, fp=fp, indent=4, sort_keys=True)